In [2]:
! git clone https://github.com/gyyang/neurogym.git
%cd neurogym/
! pip install -e .

Cloning into 'neurogym'...
remote: Enumerating objects: 11100, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 11100 (delta 132), reused 234 (delta 115), pack-reused 10827
Receiving objects: 100% (11100/11100), 8.20 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (8318/8318), done.
/home/maria/Neuromatch2024/neurogym
Obtaining file:///home/maria/Neuromatch2024/neurogym
  Preparing metadata (setup.py) ... done
  Running setup.py develop for neurogym

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import neurogym as ngym
from neurogym import spaces
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env


class ReactionTimeTask(ngym.TrialEnv):
    def __init__(self, dt=100):
        super().__init__(dt=dt)
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,))
        self.current_stimulus = 0

    def _new_trial(self, **kwargs):
        trial = {'stimulus': np.random.choice([0, 1])}
        self.current_stimulus = trial['stimulus']
        return trial

    def _step(self, action):
        reward = 0
        if self.current_stimulus == 1 and action == 1:
            reward = 1
        done = True
        obs = np.array([self.current_stimulus])
        return obs, reward, done, {}

class GoNoGoTask(ngym.TrialEnv):
    def __init__(self, dt=100):
        super().__init__(dt=dt)
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,))
        self.current_stimulus = 0

    def _new_trial(self, **kwargs):
        trial = {'stimulus': np.random.choice([0, 1])}
        self.current_stimulus = trial['stimulus']
        return trial

    def _step(self, action):
        reward = 0
        if self.current_stimulus == 1 and action == 1:
            reward = 1
        elif self.current_stimulus == 0 and action == 0:
            reward = 1
        done = True
        obs = np.array([self.current_stimulus])
        return obs, reward, done, {}

class HierarchicalTask(ngym.CompoundTask):
    def __init__(self, dt=100):
        sub_tasks = [ReactionTimeTask(dt=dt), GoNoGoTask(dt=dt)]
        super().__init__(sub_tasks, dt=dt)

    def _new_trial(self, **kwargs):
        trial = self.choose_subtask()
        return trial

    def _step(self, action):
        obs, reward, done, info = self.subtasks[self.current_task]._step(action)
        return obs, reward, done, info

# Instantiate the environment
env = HierarchicalTask(dt=100)

# Check if the environment follows the Gym API
check_env(env)


ModuleNotFoundError: No module named 'neurogym'